In [1]:
# 매 강의 시작시마다 동일하게 처리해야 함
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:13pt;
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:15px;
</style>
"""))

<b><font size="5" color="red">ch11. 데이터프레임과 시리즈</font></b>

# 9절 데이터 그룹화 및 집계

In [2]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby
cf.pivot_table

In [3]:
# 1) 그룹화 방법 1.
#      df.groupby(by=[그룹핑할 열]). 기초통계함수()[함수적용해서 출력할 열들].기초통계함수()
#                [함수적용해서 출력할 열들] 생략시 그룹핑할 열을 제외한 모든 열 적용
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
7,5.0,3.4,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
52,6.9,3.1,4.9,1.5,versicolor
57,4.9,2.4,3.3,1.0,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
110,6.5,3.2,5.1,2.0,virginica
127,6.1,3.0,4.9,1.8,virginica


In [4]:
# 잘 모르겠다~ 싶으면 요 부분만 잘 알고 있어도 됨

iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']]
iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean()

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [5]:
iris_grouped_sum = iris.groupby('Species').median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [6]:
iris_grouped_sum.loc[['setosa','versicolor']]


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [7]:

iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [8]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') | 
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [9]:
#     df.groupby(by=[그룹핑할 열])[함수적용해서 출력할 열들].함수()
# 2) 그룹화 방법 2
#     df.pivot_table(index=[그룹핑할열],
#                    values=[함수적용해서 출력할 일들],
#                    aggfunc=함수(기본값:mean))

In [10]:

iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length', 'Petal.Width'],
                aggfunc='mean')

C:\Users\Admin\AppData\Local\Temp\ipykernel_5928\2945899299.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()


,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [11]:
iris.groupby('Species').count() # 결측치를 제외한 값의 갯수
iris.pivot_table(index='Species',
                #values=['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width'],
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [12]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## 다중열로 구룹화

In [13]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [14]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [15]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [16]:
# 각열의 임의의 데이터에 결측치를 할당
# iris.iloc[0, 0] = np.nan

In [17]:
import random
print(random.sample(range(149), 3))
iris.iloc[random.sample(range(149), 3), 0]

[49, 79, 16]


11    4.8
63    6.1
0     5.1
Name: Sepal.Length, dtype: float64

In [18]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  150 non-null    float64
 1   Sepal.Width   150 non-null    float64
 2   Petal.Length  150 non-null    float64
 3   Petal.Width   150 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [19]:
iris.groupby(by=['Species','num'], 
            # as_index=False
            )[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           25
versicolor 2              25           25
           3              25           25
virginica  4              25           25
           5              25           25

In [20]:

iris.pivot_table(index=['Species', 'num'],
                values = ['Petal.Length', 'Petal.Width'],
                aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           25
versicolor 2              25           25
           3              25           25
virginica  4              25           25
           5              25           25

In [21]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
Petal.Length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
Petal.Width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [22]:
iris.drop('num', axis=1, inplace=True)

In [23]:
# 레이블을 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- 판다스 공식 문서의 melt : https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7p) : https://vita.had.co.nz/papers/tidy-data.pdf  

In [24]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                   package='datasets').data
airquality.head(2)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2


## 10.1 melt를 이용한 언피벗팅(와이드포맷 -> 롱포맷)

In [25]:
air_melted =  airquality.melt(id_vars=['Month','Day'],
               var_name='변수명', # 기본값:variable
               value_name='값') # 기본값 : value
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [26]:
# melt 연습하기 (부동산 와이드포맷 데이터를 롱포맷)
%ls c:\ai_x\download\shareData\부동산_250213\

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 c:\ai_x\download\shareData\부동산_250213 디렉터리

2025-04-30  오후 02:52    <DIR>          .
2025-04-30  오후 02:52    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 02:04           419,340 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             421,503 바이트
               2개 디렉터리  146,107,633,664 바이트 남음


In [27]:
import pandas as pd
df = pd.read_csv(r'c:\ai_x\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding='cp949')
ㅐ


NameError: name 'ᅢ' is not defined

In [ ]:

new_df = df.melt(id_vars='지역',
           var_name='연도월',
           value_name='평당분양가')
new_df.head()

## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷) 

In [ ]:
# air_meltied를 와이드 포맷으로
air_melted.head()

In [ ]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                      columns='변수명',
                      values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None


In [ ]:
airquality2.head()

# 11절. 데이터프레임(시리즈)에 함수 적용시키기
- apply(func, axis) : 축요소별(요소별) 함수 적용(데이터 프레임이나 시리즈에서 가능)
- applymap(func) : 각 요소별ㄹ 함수 적용(데이터 프레임에서만 가능)
- map(func) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)

In [ ]:
new_df.head()

## 11.1 apply

In [ ]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

In [ ]:
def get_year(datestr):
    return int(datestr.split('-')[0])

In [ ]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [ ]:
get_year('1999-05-01')

In [ ]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [ ]:
get_month('1999-05-01')

In [ ]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

In [ ]:
X = iris.iloc[:, :-2]
# X의 열별 평균
avg = X.mean(axis=0)
avg

In [ ]:
X.iloc[0] -avg

In [ ]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)
# X.apply(lambda x: (x - avg).round(2), axis=1)

## 11.2 applymap

In [ ]:
X = iris.iloc[:, :-2]

In [ ]:
X.applymap(np.round)

## 11.3 map
- 1차원

In [ ]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

In [ ]:
# 연습하기(appy, map)
# new_df의 연도월 데이터를 연도와월을 int로 분리하여 컬럼을 추가하세요
new_df.sample()

In [ ]:
new_df['연도'] = new_df['연도월'].apply(lambda str:int(str.split('년')[0]))
new_df['월'] = new_df['연도월'].map(lambda str:int(str.split('년')[1][:-1]))
new_df.info()

In [ ]:
new_df.sample()

In [ ]:
new_df.drop(['연도','월'], axis=1, inplace=True)

In [ ]:
# ★ 2차원 데이터 프레임의 apply 이용하는 방법

In [ ]:
new_df.head(2)

In [ ]:
def year_month_append(df):
    'df를 받아 연도와 월 추가후 reutrn'
    df = df.copy() # 깊은 복사
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [ ]:
year_month_append(new_df.iloc[0])

In [ ]:
new_df = new_df.apply(year_month_append, axis=1)
new_df.head()

# 12절. 일괄변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [ ]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                 columns=[list('ABCD')])
df

In [ ]:
# 모든 결측치를 0 대체
df.fillna(0)

In [ ]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

In [ ]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

In [ ]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80, 'C':75, 'D':90}
df.fillna(value=values)

In [ ]:
# 결측치를 열별 평균값으로 대체
df.mean(axis=0)
df.fillna(value=df.mean(axis=0))

In [ ]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

In [ ]:
# ★ ★ ★ 총괄 QUIZ ★ ★ ★
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당한다
        # 결측치가 각 열마다 20개씩인지 확인
# (2-1단계) 결측치는 각 열의 평균으로 대체한다 : fillna사용
# (2-2단계) 결측치는 종별 열별 평균(groupby, pivot_table)로 대체한다 : fillna사용X

In [30]:
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당한다.
#        결측치는 각 열마다 20개씩 확인


NameError: name 'collen' is not defined

In [ ]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')
rowlen, collen = iris.shape
rowlen, collen


In [ ]:
# (1단계) iris 각 열마다 20개씩 인위적으로 결측치를 할당후 확인
print('임의의 20개 index 추출 :',random.sample(range(rowlen), 20))

In [31]:
# 결측치 할당
for col in range(collen-1):
    iris.iloc[random.sample(range(rowlen), 20), col] = np.nan


Sepal.Length    0
Sepal.Width     0
Petal.Length    0
Petal.Width     0
Species         0
target          0
dtype: int64

In [ ]:
# 결측치 확인 - iris.isna(), iris.isnull():결측치인지 여부
# iris.isna().sum()
print(iris.isnull().sum())
iris.tail()


In [ ]:
# 결측치가 하나라도 있는 행 출력
iris[iris['sepal_length'].isna() | iris['sepal_width'].isna() | 
     iris['petal_length'].isna() | iris['petal_width'].isna()]


In [ ]:
# (2-1단계) 결측치는 각 열의 평균으로 대체한다 : fillna사용
#iris.mean(numeric_only=True)
#iris.iloc[:, :-1].mean()
iris.fillna(iris.iloc[:, :-1].mean())

In [ ]:
iris.isna().sum()

In [ ]:
# (2-2단계) 결측치는 종별 열별 중위수(groupby, pivot_table)로 대체한다. : fillna 사용 안댐

In [ ]:
# (2-2단계) 결측치는 종별 열별 평균(groupby, pivot_table)로 대체한다 : fillna사용X
# 종별 열별 중위수
iris_mean = iris.groupby('species').mean()
iris_mean

In [ ]:
# fillna사용X 확인
iris.fillna(iris_mean)

In [ ]:
iris.iloc[2]

In [ ]:
iris_mean # iris_mean.loc['setosa','sepal_length']

In [ ]:
# iris 한행을 매개변수로 받아 결측치가 있는 경우 종별 열별 중위수로 대체하는 함수
def iris_fillna(irisrow):
    irisrow = irisrow.copy()
    for col, value in irisrow.items():
        if isinstance(value, (int, float, np.number)) and np.isnan(value):
            # print(irisrow[col], '이 결측치')
            irisrow[col] = iris_mean.loc[irisrow.species, col]
    return irisrow


In [ ]:
iris_fillna(iris.iloc[2])

In [ ]:
# apply 적용
iris = iris.apply(iris_fillna, axis=1)

## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용
- 데이터 프레임, 시리즈 적용

In [ ]:
s = pd.Series([0,1,2,3,np.nan])
s.replace(np.nan, 0)
s.replace([0,1], 99) # 0이나 1을 99로 변경
s.replace([0,1], [100, 101]) # 0은 100으로, 1은 101로 변경


In [37]:
df = pd.DataFrame({'A': ['bat', 'foo', 'bait'],
                   'B': ['abc', 'bar', 'xyz']})
df.replace(r'ba.+', 'XX', regex=True)


,A,B
0,XX,abc
1,foo,XX
2,XX,xyz


## 12.3 mask(조건), where(조건)
- where(조건) : 조건이 True인 요소만 출력하고, 그 외 요소는 결측치로 출력
- mask(조건) : 조건이 False인 요소만 출력하고, 그 외 요소는 결측치로 출력

In [38]:
s = pd.Series(range(10,20))
s


0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
dtype: int64

In [39]:
s.where(s>15)


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64

In [40]:
s.mask(s>15)


0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

## 12.4 dropna
- 결측치가 있는 데이터 누락(행, 열)

In [41]:
df = pd.DataFrame({'name':['홍','김','이', np.nan],
                   'age': [np.nan, 20, 30, np.nan],
                   'born':['서울','부산',np.nan,np.nan]})
df


,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN
3,NaN,NaN,NaN


In [42]:
df.dropna(how='all') # 모두 결측치인 행만 삭제 (axis=0기본값)

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN


In [43]:
df.dropna() # 결측치가 하나라도 있는 행 삭제


,name,age,born
1,김,20.0,부산


In [44]:
df.dropna(axis=1) # 결측치가 하나라도 있는 열 삭제

""
0
1
2
3


In [45]:
df.dropna(subset=['name','age'])


,name,age,born
1,김,20.0,부산
2,이,30.0,NaN


# 13절 시리즈
- 1차원 데이터

In [46]:
s = pd.Series([1000, 7000, 5000],
             index=['mango','apple','banana'])
s


mango     1000
apple     7000
banana    5000
dtype: int64

In [47]:
s['mango'], s.mango

(1000, 1000)

In [48]:
s['mango'] = 2800  # 수정


In [49]:
s['mango'] = 2800  # 수정
s['orange'] = 4000 # 추가
s

mango     2800
apple     7000
banana    5000
orange    4000
dtype: int64

In [50]:
s['mango':'banana'] # 슬라이싱 [from:to] from부터 to까지(to포함)

mango     2800
apple     7000
banana    5000
dtype: int64

In [51]:
s[ ['mango','banana'] ]

mango     2800
banana    5000
dtype: int64

In [52]:
s.drop('banana')

mango     2800
apple     7000
orange    4000
dtype: int64

In [53]:
s.sort_index()

apple     7000
banana    5000
mango     2800
orange    4000
dtype: int64

In [62]:
s.sort_values() # 오름차순정렬

mango     2800
orange    4000
banana    5000
apple     7000
dtype: int64

In [63]:
s.sort_values(ascending=False) # 내림차순

apple     7000
banana    5000
orange    4000
mango     2800
dtype: int64

In [64]:
s.to_frame().T # s시리즈를 데이터프레임으로 변환하여 전치시킴

,mango,apple,banana,orange
0,2800,7000,5000,4000


# 14절 연습문제 - 실습형

In [54]:
from seaborn import load_dataset
iris = load_dataset('iris')
type(iris)

pandas.core.frame.DataFrame

In [55]:
# 1. 처음 다섯개 행만 출력


In [ ]:
# 2. iris데이터를 독립변수 x와 종속변수 y로. 종속변수는 species. x, y 모두 데이터프레임

In [ ]:
# 3. iris 데이터에서 처음 50개 행을 빼내서 temp 변수에 저장하세요

In [56]:
# 4. 3번에서 서택한 데이터프레임의 요약정보를 출력하세요. 모든 열에 대한 요약정보가 출력

In [57]:
# 5. versicolor종의 데이터만 iris_versicolor 변수에 저장하세요

In [58]:
# 6. 2번의 x와 y 변수를 합해서 iris_df 데이터 프레임으로 만드세요

In [59]:
# 7. iris 열평균

In [60]:
# 8. iris열들 사이의 상관계수

In [61]:
# 9. 변수별 평균 차이를 출력

In [ ]:
# 10. 종별 평균

In [ ]:
# 11. 각 요소들과 종별 변수의 평균과의 차이를 각 종별로 3개씩 출력